In [0]:
!rm -rf ./RL_2020
!git clone https://github.com/HanyangTechAI/RL_2020.git
!rm -rf ./RL_2020/.git

In [0]:
# 필요한 패키지를 설치해준다.
!cat ./RL_2020/lab02_deep_reinforcement_learning/requirements.txt
!echo '----------------------------------'
!pip install -r ./RL_2020/lab02_deep_reinforcement_learning/requirements.txt

## 큐 함수를 근사하는 신경망
CartPole-v0는 observation으로 네 개의 값이 들어오고, 에이전트가 할 수 있는 행동은 두 개 이다. 따라서 우리가 만들 신경망은 입력이 네 개이고 출력이 두 개여야 한다.

In [0]:
# PyTorch를 사용하기 위한 모듈을 불러온다
import torch
from torch import nn, optim

In [0]:
# 아래 ???을 채워봅시다.

class QNetwork(nn.Module):
  def __init__(self):
    super(QNetwork, self).__init__()

    self.fc1 = nn.Linear(4, 16)
    self.fc2 = nn.Linear(16, 2)

    self.act = nn.Sigmoid()

  def forward(self, x):
    x = self.act(self.fc1(x))

    return self.fc2(x)

In [0]:
import random
import numpy as np

# Q-Learning를 이용한 에이전트를 만들자.
class Agent:
  def __init__(self, actions):
    # 할 수 있는 행동의 집합
    self.actions = actions

    # 학습률로, 한 번에 얼마나 학습할지 결정한다.
    self.learning_rate = 0.001

    # 감가율
    self.discount_factor = 0.9

    # 무작위로 행동을 선택할 비율이다.
    # 강화학습에선 탐험이 중요하기 때문에 무작위 선택도 넣는다.
    self.epsilon = 0.1

    # 큐 함수를 근사할 신경망이다.
    self.q_net = QNetwork()

    # 신경망을 최적화 할 optimizer
    self.opt = optim.Adam(self.q_net.parameters(), lr=self.learning_rate)

    # 신경망을 최적화 할 때 사용할 loss function
    self.criterion = nn.MSELoss()

  # state를 받아 할 행동을 구하는 메소드
  def get_action(self, state):
    if random.random() < self.epsilon:
      # 일정한 확률로 임의의 행동을 한다.
      return self.actions.sample()
    else:
      q_values = self.q_net(state)
      return q_values.argmax(dim=0).item()

  # 보상을 바탕으로 Q Network를 업데이트 하는 메소드
  def train(self, state, action, reward, next_state):
    self.opt.zero_grad()

    q1 = self.q_net(state)[action]
    q2 = reward + self.discount_factor * torch.max(self.q_net(next_state).detach())

    loss = self.criterion(q1, q2)
    loss.backward()

    self.opt.step()

In [0]:
import gym

# 위에서 만든 에이전트를 바탕으로 학습을 시켜보자.
env = gym.make('CartPole-v0')
agent = Agent(env.action_space)

# reward를 저장할 리스트
rewards = []

# 5000 에피소드를 수행한다.
for episode in range(1, 5000 + 1):
  state = env.reset()
  
  # state를 PyTorch의 Tensor형으로 변환해야한다.
  state = torch.FloatTensor(state)

  total_reward = 0

  while True:
    action = agent.get_action(state)
    next_state, reward, done, _ = env.step(action)

    total_reward += reward

    # next_state도 Tensor형으로 변환해야한다.
    next_state = torch.FloatTensor(next_state)

    if done:
      rewards.append(total_reward)

      reward = -1
      agent.train(state, action, reward, next_state)
      break
    else:
      agent.train(state, action, reward, next_state)

    state = next_state

  # 이전 50개 에피소드의 평균 reward가 190보다 크거나 같으면 학습 중단
  if np.mean(rewards[-min(50, len(rewards)):]) >= 190:
    print('Early stopped (after {} episodes)'.format(episode))
    break

In [0]:
# 학습이 진행됨에 따른 reward 변화를 그래프로 그린다.
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(range(1, len(rewards) + 1), rewards)
plt.show()